In [ ]:
## Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install text-unidecode

# ## Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[asr]

## Install TorchAudio
!pip install torchaudio -f https://download.pytorch.org/whl/torch_stable.html

In [56]:
import os
import wget
import json
from tqdm import tqdm
from omegaconf import OmegaConf
from nemo.collections.asr.models.msdd_models import NeuralDiarizer

In [20]:
MODEL_CONFIG = 'diar_infer_telephonic.yaml'
if not os.path.exists(MODEL_CONFIG):
    config_url = "https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/speaker_tasks/diarization/conf/inference/diar_infer_telephonic.yaml"
    MODEL_CONFIG = wget.download(config_url)

config = OmegaConf.load(MODEL_CONFIG)
# print(OmegaConf.to_yaml(config))

In [61]:
config.diarizer.speaker_embeddings.parameters.window_length_in_sec = [1.5,1.25,1.0,0.75,0.5]
config.diarizer.speaker_embeddings.parameters.shift_length_in_sec = [0.75,0.625,0.5,0.375,0.1]
config.diarizer.speaker_embeddings.parameters.multiscale_weights= [1,1,1,1,1]

config.diarizer.msdd_model.model_path = 'diar_msdd_telephonic' # Telephonic speaker diarization model
config.diarizer.msdd_model.parameters.sigmoid_threshold = [0.7, 1.0] # Evaluate with T=0.7 and T=1.0

config.num_workers = 1 # Workaround for multiprocessing hanging with ipython issue

config.diarizer.speaker_embeddings.model_path = 'titanet_large'
config.diarizer.oracle_vad = False # compute VAD provided with model_path to vad config
config.diarizer.clustering.parameters.oracle_num_speakers=False

config.diarizer.vad.model_path = 'vad_multilingual_marblenet'
config.diarizer.vad.parameters.onset = 0.8
config.diarizer.vad.parameters.offset = 0.6
config.diarizer.vad.parameters.pad_offset = -0.05

config.diarizer.manifest_filepath = 'input_manifest.json'
config.diarizer.out_dir = 'outputs'

In [62]:
def nemo_diarize(path):
    base_name = os.path.splitext(os.path.basename(path))[0]
    if os.path.isfile('input_manifest.json'):
        os.system('rm input_manifest.json')
    if os.path.isdir('outputs'):
        os.system('rm -r outputs')
    meta = {
        'audio_filepath': path,
        'offset': 0,
        'duration': None,
        'label': 'infer',
        'text': '-',
        'num_speakers': 2,
        'rttm_filepath': None,
        'uem_filepath' : None
    }
    with open('input_manifest.json','w') as fp:
        json.dump(meta, fp)
        fp.write('\n')

    config.diarizer.manifest_filepath = 'input_manifest.json'
    config.diarizer.out_dir = 'outputs'
    system_vad_msdd_model = NeuralDiarizer(cfg=config)
    system_vad_msdd_model.diarize()

    rttm = []
    with open(f"outputs/pred_rttms/{base_name}.rttm", 'r') as f:
        for line in f.readlines():
            line = line.strip().split()
            rttm.append((float(line[3]), float(line[3]) + float(line[4]), line[7]))
    return rttm

In [63]:
out_dir = "timestamps_nemo_pretrained"
os.mkdir(out_dir)

src_dir = "primock57/output/mixed_audio/"
for audio in tqdm(os.listdir(src_dir)):
    uri = audio[:-4]
    if not uri in ["day5_consultation07", "day5_consultation08", "day5_consultation09", "day5_consultation10", "day5_consultation11", "day5_consultation12"]:
        continue
    audio_path = os.path.join(src_dir, audio)
    diarization = nemo_diarize(audio_path)
    with open(os.path.join(out_dir, uri + ".txt"), 'w') as f:
        for st, en, spk_id in diarization:
            f.write(f"{st:.3f},{en:.3f},{spk_id}\n")


  0%|          | 0/57 [00:00<?, ?it/s]

[NeMo I 2024-01-15 18:45:42 msdd_models:1092] Loading pretrained diar_msdd_telephonic model from NGC
[NeMo I 2024-01-15 18:45:42 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.21.0rc0/diar_msdd_telephonic/3c3697a0a46f945574fa407149975a13/diar_msdd_telephonic.nemo.
[NeMo I 2024-01-15 18:45:42 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.21.0rc0/diar_msdd_telephonic/3c3697a0a46f945574fa407149975a13/diar_msdd_telephonic.nemo
[NeMo I 2024-01-15 18:45:42 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2024-01-15 18:45:43 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    emb_dir: null
    sample_rate: 16000
    num_spks: 2
    soft_label_thres: 0.5
    labels: null
    batch_size: 15
    emb_batch_size: 0
    shuffle: true
    
[NeMo W 2024-01-15 18:45:43 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    emb_dir: null
    sample_rate: 16000
    num_spks: 2
    soft_label_thres: 0.5
    labels: null
    batch_size: 15
    emb_batch_size: 0
    shuffle: false
    
[NeMo W 2024-01-15 18:45:43 modelPT:174] Please call the ModelPT.setup_test_data() or ModelPT.setup_multiple

[NeMo I 2024-01-15 18:45:43 features:289] PADDING: 16
[NeMo I 2024-01-15 18:45:43 features:289] PADDING: 16
[NeMo I 2024-01-15 18:45:44 save_restore_connector:249] Model EncDecDiarLabelModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.21.0rc0/diar_msdd_telephonic/3c3697a0a46f945574fa407149975a13/diar_msdd_telephonic.nemo.
[NeMo I 2024-01-15 18:45:44 features:289] PADDING: 16
[NeMo I 2024-01-15 18:45:44 clustering_diarizer:127] Loading pretrained vad_multilingual_marblenet model from NGC
[NeMo I 2024-01-15 18:45:44 cloud:58] Found existing object /root/.cache/torch/NeMo/NeMo_1.21.0rc0/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo.
[NeMo I 2024-01-15 18:45:44 cloud:64] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.21.0rc0/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo
[NeMo I 2024-01-15 18:45:44 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2024-01-15 18:45:44 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/ami_train_0.63.json,/manifests/freesound_background_train.json,/manifests/freesound_laughter_train.json,/manifests/fisher_2004_background.json,/manifests/fisher_2004_speech_sampled.json,/manifests/google_train_manifest.json,/manifests/icsi_all_0.63.json,/manifests/musan_freesound_train.json,/manifests/musan_music_train.json,/manifests/musan_soundbible_train.json,/manifests/mandarin_train_sample.json,/manifests/german_train_sample.json,/manifests/spanish_train_sample.json,/manifests/french_train_sample.json,/manifests/russian_train_sample.json
    sample_rate: 16000
    labels:
    - background
    - speech
    batch_size: 256
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: sca

...

[NeMo W 2024-01-15 19:25:01 der:185] Check if each ground truth RTTMs were present in the provided manifest file. Skipping calculation of Diariazation Error Rate


[NeMo I 2024-01-15 19:25:01 speaker_utils:93] Number of files to diarize: 1


[NeMo W 2024-01-15 19:25:01 der:185] Check if each ground truth RTTMs were present in the provided manifest file. Skipping calculation of Diariazation Error Rate


[NeMo I 2024-01-15 19:25:01 msdd_models:1431]   
    


100%|██████████| 57/57 [39:19<00:00, 41.39s/it]
